<a href="https://colab.research.google.com/github/sipocz/Colab/blob/main/_earthquake_distance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#_Dengue

_PCVERSION_=False


if _PCVERSION_:
    basedir="C:/Users/sipocz/OneDrive/Dokumentumok/GitHub"
else:
    from google.colab import drive
    drive.mount('/content/drive',force_remount=True)
    basedir="/content/drive/My Drive/001_AI"

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#--------------scikit import 
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

#--------------
def outlierStatistic(X_train_predict):
    print(X_train_predict)
    maxX=len(X_train_predict)
    outlier=0
    for i in X_train_predict:
        if i==-1:
            outlier+=1
    print(f"A összes ({maxX} darabból {outlier} darab outlier van. Az {outlier/maxX*100:5.1f} %.)")


def checkvalues(df,columnname,key):
    print(f"{columnname} ellenőrzése !")
    numok=0
    numerr=0
    for i in df.index:
        if df.at[i,columnname] in key:
            #print(df.at[i,columnname])
            numok+=1
            pass
        else:
            numerr+=1
            print(df.at[i,columnname],end=", ")
    sumall=numok+numerr
    print(f"\n{sumall} mintából {numerr} db nem volt megfelelő")


def createcolumn(df,columnname,keys):
    print(f"{columnname} cseréje megy")
    for key in keys:
        df[keys[key]]=0.0
    for key in keys:
        for i in df.index:
            if df.at[i,columnname]==key:
                df.at[i,keys[key]]=1.0


def create_dict(idx,list):
    o={}
    for i in list:
        o[i]=idx+"_"+i
    return o

def create_base_data(df):
    t=['n', 't', 'o']
    columnname="land_surface_condition"
    key=create_dict(columnname,t)


    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t= ['h', 'w', 'i', 'r', 'u']
    columnname="foundation_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=  ['q', 'n', 'x']
    columnname="roof_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=  ['z', 'v', 'f', 'm', 'x']
    columnname="ground_floor_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['q', 's', 'j', 'x']
    columnname="other_floor_type"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['j', 's', 't', 'o']
    columnname="position"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=   ['c', 's', 'f', 'd', 'm', 'a', 'q', 'u', 'n', 'o']
    columnname="plan_configuration"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    t=['a', 'w', 'r', 'v']
    columnname="legal_ownership_status"
    key=create_dict(columnname,t)

    #checkvalues(df,columnname,key)
    createcolumn(df,columnname,key)

    df["geopos"]=df.geo_level_1_id+df.geo_level_2_id/10000+df.geo_level_3_id/1000000000
    return(df)



def kill_columns(df):
    notkey=["building_id","legal_ownership_status","geo_level_1_id",	"geo_level_2_id",	"geo_level_3_id", "land_surface_condition",	"foundation_type",	"roof_type",	"ground_floor_type",	"other_floor_type",	"position",	"plan_configuration"]
    for i in df.columns:
        if i in notkey:
            df.drop(columns=[i], inplace=True)
    return df

f=open(basedir+"/_EarthQuake/similarity.csv","a")
X_train_ok=pd.read_csv(basedir+"/_EarthQuake/X_tran_ok.csv")
X_pred_ok=pd.read_csv(basedir+"/_EarthQuake/X_pred_ok.csv")
y_train_ok=pd.read_csv(basedir+"/_EarthQuake/y_train_ok.csv")

X_train_ok.head()
X_pred_ok.head()
print("hello")





In [3]:
X_pred_ok.head()

,Unnamed: 0,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other,land_surface_condition_n,land_surface_condition_t,land_surface_condition_o,foundation_type_h,foundation_type_w,foundation_type_i,foundation_type_r,foundation_type_u,roof_type_q,roof_type_n,roof_type_x,ground_floor_type_z,ground_floor_type_v,ground_floor_type_f,ground_floor_type_m,ground_floor_type_x,other_floor_type_q,other_floor_type_s,other_floor_type_j,other_floor_type_x,position_j,position_s,position_t,position_o,plan_configuration_c,plan_configuration_s,plan_configuration_f,plan_configuration_d,plan_configuration_m,plan_configuration_a,plan_configuration_q,plan_configuration_u,plan_configuration_n,plan_configuration_o,legal_ownership_status_a,legal_ownership_status_w,legal_ownership_status_r,legal_ownership_status_v,geopos
0,0,3,20,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.059611
1,1,2,25,13,5,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,6.014112
2,2,2,5,4,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,22.001910
3,3,1,0,19,3,0,0,0,0,0,1,0,0,0,0,0,2,1,0,0,1,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,26.003901
4,4,3,15,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,17.028908


In [13]:
X_train_ok.loc[4][1:]

count_floors_pre_eq              3.000000
age                             30.000000
area_percentage                  8.000000
height_percentage                9.000000
has_superstructure_adobe_mud     1.000000
                                  ...    
legal_ownership_status_a         0.000000
legal_ownership_status_w         0.000000
legal_ownership_status_r         0.000000
legal_ownership_status_v         1.000000
geopos                          11.013101
Name: 4, Length: 66, dtype: float64

In [10]:
np.linalg.norm(X_train_ok.loc[4][1:])

34.428017723723045

In [20]:
X_pred_len=[]

for i in X_pred_ok.index:
    a=np.linalg.norm(X_pred_ok.loc[i][1:])
    X_pred_len.append(a)





In [21]:
X_train_len=[]

for i in X_pred_ok.index:
    a=np.linalg.norm(X_train_ok.loc[i][1:])
    X_train_len.append(a)

In [ ]:
'''
f=open(basedir+"/_EarthQuake/difference.csv","a")
epsilon=0.001
k=0
for i,Xpi in enumerate(X_pred_len):
    for j,Xti  in enumerate(X_train_len):
        if abs(Xpi-Xti)<epsilon:
            delta=np.linalg.norm(X_pred_ok.loc[i][1:]-X_train_ok.loc[j][1:])
            if delta<6:
                k+=1
                if k>500:
                    print(i,j)
                    k=0
                print (f"{i},{j},{delta}", file=f)
f.close()
'''            

In [60]:
type(X_train_ok)
y_train_ok

,Unnamed: 0,damage_grade
0,0,3
1,1,2
2,2,3
3,3,2
4,4,3
...,...,...
260596,260596,2
260597,260597,3
260598,260598,3
260599,260599,2


In [61]:
X_train1=[]
y_train1=[]
f=open(basedir+"/_EarthQuake/difference.csv","r")
for row in f:
    a=row.split(",")
    inx=int(a[1])
    X_train1.append(X_train_ok.loc[inx])
    #y_train1.append(y_train_ok[inx])        

In [62]:
X_train1

[Unnamed: 0                  15506.000000
 count_floors_pre_eq             3.000000
 age                            20.000000
 area_percentage                 6.000000
 height_percentage               7.000000
                                 ...     
 legal_ownership_status_a        0.000000
 legal_ownership_status_w        0.000000
 legal_ownership_status_r        0.000000
 legal_ownership_status_v        1.000000
 geopos                         17.002111
 Name: 15506, Length: 67, dtype: float64,
 Unnamed: 0                  49275.000000
 count_floors_pre_eq             3.000000
 age                            20.000000
 area_percentage                 7.000000
 height_percentage               6.000000
                                 ...     
 legal_ownership_status_a        0.000000
 legal_ownership_status_w        0.000000
 legal_ownership_status_r        0.000000
 legal_ownership_status_v        1.000000
 geopos                         17.059611
 Name: 49275, Length: 67, dtype: f

In [64]:
y_train1=[]
f=open(basedir+"/_EarthQuake/difference.csv","r")
for row in f:
    a=row.split(",")
    inx=int(a[1])
    y_train1.append(y_train_ok.loc[inx])

In [68]:
y_train_df=pd.DataFrame(y_train1)
X_train_df=pd.DataFrame(X_train1)

In [74]:
def kill_columns(df):
    notkey=["Unnamed: 0","building_id","legal_ownership_status","geo_level_1_id",	"geo_level_2_id",	"geo_level_3_id", "land_surface_condition",	"foundation_type",	"roof_type",	"ground_floor_type",	"other_floor_type",	"position",	"plan_configuration"]
    for i in df.columns:
        if i in notkey:
            df.drop(columns=[i], inplace=True)
    return df


In [78]:
X_train_df=kill_columns(X_train_df)
y_train_df=kill_columns(y_train_df)

In [79]:
X_train_df.columns

Index(['count_floors_pre_eq', 'age', 'area_percentage', 'height_percentage',
       'has_superstructure_adobe_mud', 'has_superstructure_mud_mortar_stone',
       'has_superstructure_stone_flag',
       'has_superstructure_cement_mortar_stone',
       'has_superstructure_mud_mortar_brick',
       'has_superstructure_cement_mortar_brick', 'has_superstructure_timber',
       'has_superstructure_bamboo', 'has_superstructure_rc_non_engineered',
       'has_superstructure_rc_engineered', 'has_superstructure_other',
       'count_families', 'has_secondary_use', 'has_secondary_use_agriculture',
       'has_secondary_use_hotel', 'has_secondary_use_rental',
       'has_secondary_use_institution', 'has_secondary_use_school',
       'has_secondary_use_industry', 'has_secondary_use_health_post',
       'has_secondary_use_gov_office', 'has_secondary_use_use_police',
       'has_secondary_use_other', 'land_surface_condition_n',
       'land_surface_condition_t', 'land_surface_condition_o',
       'fo

In [81]:
y_train_df.head()

,damage_grade
15506,3
49275,3
55389,3
55926,3
61299,2


In [ ]:

from xgboost import XGBClassifier  # 72.09
# max_depth=10 : 72.79857561664441

knn = XGBClassifier(verbosity=3,max_depth = 24, n_estimators=50)

print("Fit: Inlier betanitás Start--")
knn.fit(X_train_df, y_train_df)
print("Fit End - Prediction Start ")

y_pred=knn.predict(X_train_df)
print("Prediction End")

accuracy =  accuracy_score(y_pred, y_train_df) * 100
print(f"Accuracy a betanított halmazra : {accuracy}")



Fit: Inlier betanitás Start--


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[23:20:37] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 5640 extra nodes, 0 pruned nodes, max_depth=24
[23:20:39] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 17884 extra nodes, 0 pruned nodes, max_depth=24
[23:20:42] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 15664 extra nodes, 0 pruned nodes, max_depth=24
[23:20:44] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 5844 extra nodes, 0 pruned nodes, max_depth=24
[23:20:47] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 18540 extra nodes, 0 pruned nodes, max_depth=24
[23:20:50] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 15838 extra nodes, 0 pruned nodes, max_depth=24
[23:20:52] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 6188 extra nodes, 0 pruned nodes, max_depth=24
[23:20:55] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning e

In [85]:
X_pred_ok=kill_columns(X_pred_ok)

In [87]:
#outfile generation
y_pred_ok=knn.predict(X_pred_ok)
print("Prediction End")

X_pred_bd=pd.read_csv(basedir+"/_EarthQuake/test_values.csv")



Prediction End


In [88]:
X_pred_bd

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,300051,17,596,11307,3,20,7,6,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,99355,6,141,11987,2,25,13,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,1,1,0,0,0,0,0,0,0,0,0
2,890251,22,19,10044,2,5,4,5,t,r,n,f,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,745817,26,39,633,1,0,19,3,t,r,x,v,j,t,d,0,0,0,0,0,1,0,0,0,0,0,v,2,1,0,0,1,0,0,0,0,0,0,0
4,421793,17,289,7970,3,15,8,7,t,r,q,f,q,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86863,310028,4,605,3623,3,70,20,6,t,r,q,f,q,t,d,0,1,0,0,0,0,1,0,0,0,0,w,1,1,1,0,0,0,0,0,0,0,0,0
86864,663567,10,1407,11907,3,25,6,7,n,r,n,f,q,s,d,1,1,1,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
86865,1049160,22,1136,7712,1,50,3,3,t,r,n,f,j,s,d,0,1,0,0,0,0,1,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
86866,442785,6,1041,912,2,5,9,5,t,r,n,f,q,s,d,1,1,0,0,0,0,0,0,0,0,0,a,1,0,0,0,0,0,0,0,0,0,0,0


In [89]:
buildingid=X_pred_bd["building_id"]
head2=y_pred_ok


outdf=pd.DataFrame(data={"damage_grade":y_pred_ok} ,index=buildingid)
outdf.index.name="building_id"


outdf.head()


outdf.to_csv(basedir+"/_EarthQuake/submission_14_xgboost.csv")
print()
print(basedir+"/_EarthQuake/submission_14_xgboost.csv")


/content/drive/My Drive/001_AI/_EarthQuake/submission_13_xgboost.csv
